In [1]:
from cmeutils.structure import (
    bond_distribution,
    angle_distribution,
    dihedral_distribution,
    gsd_rdf,
    
)
from cmeutils.polymers import persistence_length
from cmeutils.plotting import get_histogram
#from msibi.utils.smoothing import savitzky_golay
#from msibi.potentials import save_table_potential
#from msibi.potentials import linear_head_correction
from polybinderCG.coarse_grain import System
from utils import radius_of_gyration, end_to_end, sample_Rg, sample_Re

import gsd.hoomd
import matplotlib.pyplot as plt
import numpy as np
import signac
from scipy.stats import sem


import os


def moving_average_smoothing(raw_data, w):
        return np.convolve(raw_data, np.ones(w), "same")/w

    
def hoomd_dihedral(phi, phi0, k, d, n):
    return 0.5*k*(1+d*np.cos(n*phi - phi0))

def bond_harmonic(l, l0, k):
    return (0.5*k*(l-l0)**2)
    
# Matplotlib RC Params:    
color_dict = {1.0: "#0F0E0F", 0.8: "#666666", 0.7: "#964d4d", 0.6: "#d03f3f"}
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["legend.fontsize"] = 7
plt.rcParams['lines.markersize'] = 1
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["lines.marker"] = "o"
plt.rcParams["lines.linewidth"] = 1
plt.rcParams["figure.dpi"] = 1200

label_conv = {
    0.60: "60/40",
    0.70: "70/30",
    0.80: "80/20",
    1.0: "100/0",
}

/home/chrisjones4/miniconda3/envs/cmeutils-dev/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/chrisjones4/miniconda3/envs/cmeutils-dev/lib/python3.11/site-packages/mdtraj/geometry/order.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/chrisjones4/miniconda3/envs/cmeutils-dev/lib/python3.11/site-packages/pkg_resources/__init__.py:2846: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/chrisjones4/miniconda3/envs/cmeutils-dev/lib/python3.11/site-packages/lark/u

In [2]:
ti_ratio = 0.6
n_frames = 500
lp_window_size = 20

In [3]:
cg_project = signac.get_project("../coarse-grain-sims/single-chain-pl-rg-re/polybinder-flow/")
cg_project_init = signac.get_project(
    "/home/erjank_project/chrisjones/pekk-msibi/coarse-grain-sims/single-chain-pl-rg-re/polybinder-flow"
)
ua_project = signac.get_project("../ua-target-runs/rg-re-pl-sweep/")

In [4]:
cg_project.detect_schema()

ProjectSchema(<len=56>)

# Independent Sampling of Rg, Re, Lp

In [5]:
for job in cg_project_init.find_jobs():
    with gsd.hoomd.open(job.fn("sim_traj.gsd"), "rb") as traj:
        lp_stop = len(traj)
        lp_start = lp_stop - n_frames

    lp, lp_std = persistence_length(
        gsd_file=job.fn("sim_traj.gsd"),
        start=lp_start,
        stop=lp_stop,
        window_size=lp_window_size,
        select_atoms_arg="name E K"
    )
    job.doc.lp_mean = lp
    job.doc.lp_std = lp_std
    
    # Radius of Gyration:
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.rg_sample = rg_sample
    job.doc.rg_mean = np.mean(rg_sample)
    job.doc.rg_std = np.std(rg_sample)
    job.doc.rg_sem = sem(rg_sample)
    
    # End-to-End Distance:
    re_sample, inds, start, ineff, Neff = sample_Re(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.re_sample = re_sample
    job.doc.re_mean = np.mean(re_sample)
    job.doc.re_std = np.std(re_sample)
    job.doc.re_sem = sem(re_sample)

/home/chrisjones4/miniconda3/envs/polybinder2/lib/python3.9/site-packages/gsd/hoomd.py:1125: FutureWarning: The 'rb' mode is deprecated, use 'r'
  gsdfileobj = fl.open(name=str(name),
/home/chrisjones4/miniconda3/envs/polybinder2/lib/python3.9/site-packages/MDAnalysis/coordinates/GSD.py:263: FutureWarning: The 'rb' mode is deprecated, use 'r'
  gsdfileobj = gsd.fl.open(name=name,
/home/chrisjones4/miniconda3/envs/polybinder2/lib/python3.9/site-packages/MDAnalysis/analysis/base.py:447: UserWarning: Reader has no dt information, set to 1.0 ps
  self.times[i] = ts.time
/home/chrisjones4/miniconda3/envs/polybinder2/lib/python3.9/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)
<__array_function__ internals>:200: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surpris

ValueError: number sections must be larger than 0.

In [ ]:
for job in cg_project.find_jobs():
    
    with gsd.hoomd.open(job.fn("sim_traj.gsd"), "rb") as traj:
        lp_stop = len(traj)
        lp_start = lp_stop - n_frames

    lp, lp_std = persistence_length(
        gsd_file=job.fn("sim_traj.gsd"),
        start=lp_start,
        stop=lp_stop,
        window_size=lp_window_size,
        select_atoms_arg="name E K"
    )
    job.doc.lp_mean = lp
    job.doc.lp_std = lp_std
    
    # Radius of Gyration:
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.rg_sample = rg_sample
    job.doc.rg_mean = np.mean(rg_sample)
    job.doc.rg_std = np.std(rg_sample)
    job.doc.rg_sem = sem(rg_sample)
    
    # End-to-End Distance:
    re_sample, inds, start, ineff, Neff = sample_Re(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.re_sample = re_sample
    job.doc.re_mean = np.mean(re_sample)
    job.doc.re_std = np.std(re_sample)
    job.doc.re_sem = sem(re_sample)

In [ ]:
kT = 4.5
ekk_weight = 0.8

for job in cg_project.find_jobs({"cg_potentials_dir": "msibi-final-2", "system_seed": 42, "kT_quench": kT}):
    if job.sp.ekk_weight != ekk_weight: continue
    
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("sim_traj.gsd"), cut_frames=10)
    plt.plot(rg_sample)

In [ ]:
fig = plt.figure()
for idx, ti in enumerate(ti_ratio):
    plt.plot(re_values[idx][5:100], label=ti)
    
plt.legend()

In [ ]:
for kT, job in cg_project.find_jobs(filter={
    "ekk_weight": ti_ratio, "cg_potentials_dir": "msibi-final-2", "system_seed": 42}).groupby("kT_quench"):
    
    print(job)

In [ ]:
for job in cg_project.find_jobs({"ekk_weight": 1.0}):
    data = np.genfromtxt(job.fn("sim_traj.txt"), names=True)
    pe = data["mdcomputeThermodynamicQuantitiespotential_energy"]/20
    print(job.id)
    plt.plot(pe[-500:])

In [ ]:
ti_ratio = 0.6
n_frames = 500
lp_window_size = 20

In [ ]:
ua_temps = []

ua_lp_results = []
ua_lp_std = []

ua_rg_results = []
ua_rg_std = []
ua_rg_sem = []

ua_re_results = []
ua_re_std = []
ua_re_sem = []

for kT, job in ua_project.find_jobs(filter={"para_weight": ti_ratio}).groupby("kT_quench"):
    job = list(job)[0]
    ua_temps.append(kT)
    
    # Persistence Lenghts
    with gsd.hoomd.open(job.fn("components.gsd"), "rb") as traj:
        lp_stop = len(traj)
        lp_start = lp_stop - n_frames
        
    lp, lp_std = persistence_length(
        gsd_file=job.fn("components.gsd"),
        start=lp_start,
        stop=lp_stop,
        window_size=lp_window_size,
        select_atoms_arg="name E K"
    )
    job.doc.lp_mean = lp
    job.doc.lp_std = lp_std
    ua_lp_results.append(lp)
    ua_lp_std.append(lp_std)
    
    # Radius of Gyration:
    
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("components.gsd"), cut_frames=10)
    job.doc.rg_sample = rg_sample
    job.doc.rg_mean = np.mean(rg_sample)
    job.doc.rg_std = np.std(rg_sample)
    job.doc.rg_sem = sem(rg_sample)
    ua_rg_results.append(job.doc.rg_mean)
    ua_rg_std.append(job.doc.rg_std)
    ua_rg_sem.append(job.doc.rg_sem)

    
    # End-to-End Distance:
    re_sample, inds, start, ineff, Neff = sample_Re(job.fn("components.gsd"), cut_frames=10)
    job.doc.re_sample = re_sample
    job.doc.re_mean = np.mean(re_sample)
    job.doc.re_std = np.std(re_sample)
    job.doc.re_sem = sem(re_sample)
    ua_re_results.append(job.doc.re_mean)
    ua_re_std.append(job.doc.re_std)
    ua_re_sem.append(job.doc.re_sem)  

In [ ]:
cg_temps = []

cg_lp_results = []
cg_lp_std = []
cg_lp_sem = []

cg_rg_results = []
cg_rg_std = []
cg_rg_sem = []

cg_re_results = []
cg_re_std = []
cg_re_sem = []


for kT, job in cg_project.find_jobs(filter={
    "ekk_weight": ti_ratio, "cg_potentials_dir": "msibi-final-2", "system_seed": 42}).groupby("kT_quench"):
    job = list(job)[0]
    cg_temps.append(kT)
    with gsd.hoomd.open(job.fn("sim_traj.gsd"), "rb") as traj:
        lp_stop = len(traj)
        lp_start = lp_stop - n_frames

    lp, lp_std = persistence_length(
        gsd_file=job.fn("sim_traj.gsd"),
        start=lp_start,
        stop=lp_stop,
        window_size=lp_window_size,
        select_atoms_arg="name E K"
    )
    cg_lp_results.append(lp)
    cg_lp_std.append(lp_std)
    
    # Radius of Gyration:
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.rg_sample = rg_sample
    job.doc.rg_mean = np.mean(rg_sample)
    job.doc.rg_std = np.std(rg_sample)
    job.doc.rg_sem = sem(rg_sample)
    cg_rg_results.append(job.doc.rg_mean)
    cg_rg_std.append(job.doc.rg_std)
    cg_rg_sem.append(job.doc.rg_sem)
    
    # End-to-End Distance:
    re_sample, inds, start, ineff, Neff = sample_Re(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.re_sample = re_sample
    job.doc.re_mean = np.mean(re_sample)
    job.doc.re_std = np.std(re_sample)
    job.doc.re_sem = sem(re_sample)
    cg_re_results.append(job.doc.re_mean)
    cg_re_std.append(job.doc.re_std)
    cg_re_sem.append(job.doc.re_sem)    

In [ ]:
cg_temps2 = []

cg_lp_results2 = []
cg_lp_std2 = []
cg_lp_sem2 = []

cg_rg_results2 = []
cg_rg_std2 = []
cg_rg_sem2 = []

cg_re_results2 = []
cg_re_std2 = []
cg_re_sem2 = []

for kT, job in cg_project.find_jobs(filter={
    "ekk_weight": ti_ratio, "cg_potentials_dir": "msibi-final-2", "system_seed": 42}).groupby("kT_quench"):
    job = list(job)[0]
    cg_temps.append(kT)
    with gsd.hoomd.open(job.fn("sim_traj.gsd"), "rb") as traj:
        lp_stop = len(traj)
        lp_start = lp_stop - n_frames

    lp, lp_std = persistence_length(
        gsd_file=job.fn("sim_traj.gsd"),
        start=lp_start,
        stop=lp_stop,
        window_size=lp_window_size,
        select_atoms_arg="name E K"
    )
    cg_lp_results.append(lp)
    cg_lp_std.append(lp_std)
    
    # Radius of Gyration:
    rg_sample, inds, start, ineff, Neff = sample_Rg(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.rg_sample = rg_sample
    job.doc.rg_mean = np.mean(rg_sample)
    job.doc.rg_std = np.std(rg_sample)
    job.doc.rg_sem = sem(rg_sample)
    cg_rg_results2.append(job.doc.rg_mean)
    cg_rg_std.append(job.doc.rg_std)
    cg_rg_sem.append(job.doc.rg_sem)
    
    # End-to-End Distance:
    re_sample, inds, start, ineff, Neff = sample_Re(job.fn("sim_traj.gsd"), cut_frames=10)
    job.doc.re_sample = re_sample
    job.doc.re_mean = np.mean(re_sample)
    job.doc.re_std = np.std(re_sample)
    job.doc.re_sem = sem(re_sample)
    cg_re_results.append(job.doc.re_mean)
    cg_re_std.append(job.doc.re_std)
    cg_re_sem.append(job.doc.re_sem)    

In [ ]:
fig = plt.figure()
plt.errorbar(x=ua_temps, y=ua_lp_results, yerr=ua_lp_std, label="Atomistic Target", c="k")
plt.errorbar(x=cg_temps, y=cg_lp_results, yerr=cg_lp_std, label="2nd Angle", c="r")
plt.errorbar(x=cg_temps2, y=cg_lp_results2, yerr=cg_lp_std2, label="1st Angle", c="g")

#plt.errorbar(x=cg2_temps, y=cg2_lp_results, yerr=cg2_lp_std, label="MSIBI: Angle Updated", c="gray")

plt.legend()
plt.xlabel("kT")
plt.ylabel("$l_p (\sigma)$")
plt.title(f"Persistence Length (T/I={label_conv[ti_ratio]})")
plt.show()

In [ ]:
fig = plt.figure()
plt.errorbar(x=ua_temps, y=ua_rg_results, yerr=ua_rg_sem, label="Atomistic Target", c="k")
plt.errorbar(x=cg_temps, y=cg_rg_results, yerr=cg_rg_sem, label="2nd Angle", c="r")
plt.errorbar(x=cg_temps2, y=cg_rg_results2, yerr=cg_rg_sem2, label="1st Angle", c="g")

plt.xlabel("kT")
plt.ylabel("$<R_g^2> (\sigma^2)$")
plt.title(f"Radius of Gyration (T/I={label_conv[ti_ratio]})")
plt.legend()

In [ ]:
fig = plt.figure()
plt.errorbar(x=ua_temps, y=ua_re_results, yerr=ua_re_sem, label="Atomistic Target", c="k")
plt.errorbar(x=cg_temps, y=cg_re_results, yerr=cg_re_sem, label="2nd Angle", c="r")
plt.errorbar(x=cg_temps2, y=cg_re_results2, yerr=cg_re_sem2, label="1st Angle", c="g")

plt.xlabel("kT")
plt.ylabel("$<R_e^2> (\sigma^2)$")
plt.title(f"End-to-End Distance (T/I={label_conv[ti_ratio]})")
plt.legend()

In [ ]:
plt.errorbar(x=ua_temps, y=np.array(ua_re_results)/np.array(ua_rg_results), label="Atomistic Target", c="k")
plt.errorbar(x=cg_temps, y=np.array(cg_re_results)/np.array(cg_rg_results), label="MSIBI-New", c="r")
plt.errorbar(x=cg_temps, y=np.array(cg_re_results2)/np.array(cg_rg_results2), label="MSIBI-Old", c="g")

plt.xlabel("kT")
plt.ylabel("$<R_e^2> / <R_g^2>$")
plt.legend()